In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-12-20 22:00:00   0.75839   0.75867  0.75835    0.75865   0.75913   
1 2020-12-20 22:05:00   0.75873   0.75929  0.75873    0.75913   0.75924   
2 2020-12-20 22:10:00   0.75934   0.75956  0.75914    0.75956   0.75981   
3 2020-12-20 22:15:00   0.75947   0.75955  0.75929    0.75955   0.75986   
4 2020-12-20 22:20:00   0.75959   0.75977  0.75959    0.75969   0.76011   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   0.75932  0.75899    0.75917   0.75876   0.75896  0.75867    0.75891  
1   0.76021  0.75924    0.76014   0.75898   0.75968  0.75898    0.75964  
2   0.75990  0.75976    0.75988   0.75958   0.75972  0.75945    0.75972  
3   0.76004  0.75985    0.76003   0.75966   0.75979  0.75958    0.75979  
4   0.76031  0.76009    0.76010   0.75985   0.76004  0.75985    0.75990  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(14).sum() / 14


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

In [6]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()

df['atr'] = atr(df)
df['rsi'] = rsi(df)
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
# df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
# df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=8).mean() - pd.Series.ewm(df['Mid_Close'], span=21).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=5).mean()
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df['sar'] = psar(df)

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2020-12-20 23:15:00   0.75954   0.75967  0.75933    0.75935   0.75972   
1     2020-12-20 23:20:00   0.75940   0.75969  0.75937    0.75955   0.75953   
2     2020-12-20 23:25:00   0.75957   0.75957  0.75926    0.75937   0.75974   
3     2020-12-20 23:30:00   0.75941   0.75941  0.75891    0.75902   0.75956   
4     2020-12-20 23:35:00   0.75905   0.75913  0.75841    0.75844   0.75924   
...                   ...       ...       ...      ...        ...       ...   
73800 2021-12-20 07:35:00   0.70846   0.70847  0.70833    0.70843   0.70859   
73801 2021-12-20 07:40:00   0.70848   0.70895  0.70843    0.70880   0.70859   
73802 2021-12-20 07:45:00   0.70877   0.70896  0.70875    0.70886   0.70891   
73803 2021-12-20 07:50:00   0.70883   0.70920  0.70883    0.70920   0.70896   
73804 2021-12-20 07:55:00   0.70916   0.70945  0.70912    0.70919   0.70928   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [18]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, spread_cutoff, use_tl, min_pips_to_risk, max_pips_to_risk, strict_macd, two_rsi, two_sar):
    max_pips_to_risk /= 10000
    min_pips_to_risk /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None
    lookback = 12

    i = lookback

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200_2, ema100_2, ema50_2, ema25_2, atr2, sar2, rsi2, mid_open2, mid_close2, mid_low2, mid_high2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, mid_open1, mid_close1, mid_low1, mid_high1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        stoch_vals = df.loc[df.index[i - 12:i], 'slowk']
        stoch_vals_2 = df.loc[df.index[i - 12:i], 'slowd']
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        emas_buy_signal = ema200_1 < mid_close1
        emas_sell_signal = ema200_1 > mid_close1
        rsi_buy_signal = rsi2 > 50 and rsi1 > 50 if two_rsi else rsi1 > 50
        rsi_sell_signal = rsi2 < 50 and rsi1 < 50 if two_rsi else rsi1 < 50
        stoch_buy_signal = min(stoch_vals) < 20 and min(stoch_vals_2) < 20
        stoch_sell_signal = max(stoch_vals) > 80 and max(stoch_vals_2) > 80
        macd_below_zero = max(macd_vals) == 0 if strict_macd else True
        macd_above_zero = min(macd_vals) == 0 if strict_macd else True
        sar_buy_signal = sar2 < min([mid_open1, mid_close1]) and sar1 < min([mid_open1, mid_close1]) if two_sar else sar1 < min([mid_open1, mid_close1])
        sar_sell_signal = sar2 > max([mid_open1, mid_close1]) and sar1 > max([mid_open1, mid_close1]) if two_sar else sar1 > max([mid_open1, mid_close1])

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and macd_below_zero and emas_buy_signal and rsi_buy_signal and sar_buy_signal and enough_volatility:
            candle_below_ema200 = True

            # for z in range(i - 4, i):
            #   mid_high, mid_low, curr_ema200 = df.loc[df.index[z], ['Mid_High', 'Mid_Low', 'ema200']]

            #   if min([mid_high, mid_low]) < curr_ema200:
            #     candle_below_ema200 = True
            #     break

            if candle_below_ema200:
              all_buys = True

              if all_buys:
                open_price = float(curr_ask_open)
                pullback = None
                # pullback = min(df.loc[df.index[i - lookback:i], 'Mid_Low'])
                pullback = sar1

                # if open_price - pullback < min_pips_to_risk:
                #   pullback = open_price - min_pips_to_risk

                if open_price - pullback > max_pips_to_risk:
                  pullback = open_price - max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and macd_above_zero and emas_sell_signal and rsi_sell_signal and sar_sell_signal and enough_volatility:
            candle_above_ema200 = True
            
            # for z in range(i - 4, i):
            #   mid_high, mid_low, curr_ema200 = df.loc[df.index[z], ['Mid_High', 'Mid_Low', 'ema200']]

            #   if max([mid_high, mid_low]) > curr_ema200:
            #     candle_above_ema200 = True
            #     break

            if candle_above_ema200:
              all_sells = True

              if all_sells:
                open_price = float(curr_bid_open)
                pullback = None
                # pullback = max(df.loc[df.index[i - lookback:i], 'Mid_High'])
                pullback = sar1

                # if pullback - open_price < min_pips_to_risk:
                #   pullback = open_price + min_pips_to_risk

                if pullback - open_price > max_pips_to_risk:
                  pullback = open_price + max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']
          # if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'buy' and curr_bid_high - trade['open_price'] >= trade['pips_risked']:
          #     trade['stop_loss'] = trade['open_price']
          #     reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
          #     day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
          #     trade['n_units'] = trade['n_units'] / 2
              

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low
          # if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'sell' and trade['open_price'] - curr_ask_low >= trade['pips_risked']:
          #     trade['stop_loss'] = trade['open_price']
          #     reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
          #     day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
          #     trade['n_units'] = trade['n_units'] / 2

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [19]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5, 2]
# risk_reward_ratio_vals = [2]
spread_cutoffs = [2, 2.5]
# spread_cutoffs = [2.5]
use_tl_vals = [True, False]
# use_tl_vals = [False]
# min_pips_to_risk_vals = [5, 10]
min_pips_to_risk_vals = [10]
max_pips_to_risk_vals = [15, 20, 25, 30]
strict_macd_vals = [True, False]
two_rsi_vals = [True, False]
two_sar_vals = [True, False]
# max_pips_to_risk_vals = [30]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_tl_vals) * len(min_pips_to_risk_vals) * len(max_pips_to_risk_vals) * len(strict_macd_vals) * len(two_rsi_vals) * len(two_sar_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for spread_val in spread_cutoffs:
    for use_tl in use_tl_vals:
      for min_pips in min_pips_to_risk_vals:
        for max_pips in max_pips_to_risk_vals:
          for strict_macd in strict_macd_vals:
            for two_rsi in two_rsi_vals:
              for two_sar in two_sar_vals:
                all_combos.append((risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, strict_macd, two_rsi, two_sar))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_tl_val = None
best_min_pips_val = None
best_max_pips_val = None
best_strict_macd_val = None
best_two_rsi_val = None
best_two_sar_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, strict_macd, two_rsi, two_sar in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, strict_macd, two_rsi, two_sar)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_tl': use_tl, 'min_pips': min_pips, 'max_pips': max_pips, 'strict_macd': strict_macd, 'two_rsi': two_rsi, 'two_sar': two_sar})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_spread_cutoff = spread_val
    best_use_tl_val = use_tl
    best_min_pips_val = min_pips
    best_max_pips_val = max_pips
    best_strict_macd_val = strict_macd
    best_two_rsi_val = two_rsi
    best_two_sar_val = two_sar

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 384

-7824.000000000001
Num buys: 391
Num sells: 428
Num trades: 819
Num wins: 217
Num losses: 601
Win streak: 5
Loss streak: 22
Remaining runs: 383
Best reward so far: -7824.000000000001

-9213.499999999894
Num buys: 458
Num sells: 499
Num trades: 957
Num wins: 279
Num losses: 674
Win streak: 7
Loss streak: 23
Remaining runs: 382
Best reward so far: -7824.000000000001

-1389.4999999999475
Num buys: 82
Num sells: 94
Num trades: 176
Num wins: 55
Num losses: 121
Win streak: 4
Loss streak: 10
Remaining runs: 381
Best reward so far: -1389.4999999999475

-3569.9999999999736
Num buys: 162
Num sells: 177
Num trades: 339
Num wins: 94
Num losses: 242
Win streak: 4
Loss streak: 17
Remaining runs: 380
Best reward so far: -1389.4999999999475

-8426.499999999894
Num buys: 447
Num sells: 488
Num trades: 935
Num wins: 274
Num losses: 657
Win streak: 6
Loss streak: 23
Remaining runs: 379
Best reward so far: -1389.4999999999475

-13801.999999999869
Num buys: 687
Num sells: 735
Num trades: 142

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best TL val: ' + str(best_use_tl_val))
print('Best min pips to risk: ' + str(best_min_pips_val))
print('Best max pips to risk: ' + str(best_max_pips_val))
print('Best strict macd val: ' + str(best_strict_macd_val))
print('Best two rsi val: ' + str(best_two_rsi_val))
print('Best two sar val: ' + str(best_two_sar_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: -616.9999999999951
Best risk/reward ratio: 2
Best spread: 2.5
Best TL val: False
Best min pips to risk: 10
Best max pips to risk: 20
-----------------------
Top results:
{'reward': -684, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 30}
{'reward': -781, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 5, 'max_pips': 30}
{'reward': -778, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 15}
{'reward': -722, 'ratio': 2, 'spread': 2.5, 'use_tl': True, 'min_pips': 10, 'max_pips': 30}
{'reward': -620, 'ratio': 2, 'spread': 2, 'use_tl': False, 'min_pips': 10, 'max_pips': 20}
{'reward': -714, 'ratio': 1.5, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 20}
{'reward': -616, 'ratio': 2, 'spread': 2.5, 'use_tl': False, 'min_pips': 10, 'max_pips': 20}
{'reward': -765, 'ratio': 2, 'spread': 2, 'use_tl': True, 'min_pips': 10, 'max_pips': 25}
{'reward': -752, 'ratio': 2, 'spread': 2.5